In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install transformers

from transformers import AutoTokenizer, AutoModel
import torch

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
sinhvien_df = pd.read_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Pre-processed/Pre-Processed Data/preprocessed_01_sinhvien.xlsx")
diem_df = pd.read_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Pre-processed/Pre-Processed Data/preprocessed_02_diem.xlsx")
totnghiep_df = pd.read_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Pre-processed/Pre-Processed Data/preprocessed_14_totnghiep.xlsx")

### Nối bảng dữ liệu

In [6]:
sinhvien_totnghiep_df = pd.merge(totnghiep_df, sinhvien_df, on='mssv', how='left')

In [7]:
diem_sinhvien_totnghiep_df = pd.merge(sinhvien_totnghiep_df, diem_df, on='mssv', how='inner')

### Chọn thuộc tính

In [8]:
feature_selection_df = diem_sinhvien_totnghiep_df[['mssv', 'gioitinh', 'khoa', 'hedt', 'nganhhoc', 'mamh', 'sotc', 'diem', 'trangthai', 'namhoc', 'hocky']]

### Lọc dữ liệu

In [9]:
filtered_df = feature_selection_df[feature_selection_df['trangthai'].isin([1, 2, 3])]

In [10]:
unique_df = filtered_df.drop_duplicates(subset=['mssv', 'namhoc', 'hocky', 'mamh'])

### Gộp dữ liệu kỳ 3 vào kỳ 2

In [11]:
diem_term_1 = unique_df[unique_df['hocky'] == 1]
diem_term_2 = unique_df[unique_df['hocky'] == 2]
diem_term_3 = unique_df[unique_df['hocky'] == 3]

In [12]:
diem_term_2_3 = pd.merge(diem_term_2, diem_term_3, on=['mssv', 'namhoc', 'mamh'], how='left')[['mssv', 'gioitinh_x', 'khoa_x', 'hedt_x', 'nganhhoc_x', 'mamh', 'sotc_x', 'diem_x', 'diem_y', 'trangthai_x', 'namhoc', 'hocky_x']]

In [13]:
diem_term_2_3['diem'] = diem_term_2_3.apply(lambda x: x['diem_y'] if not pd.isna(x['diem_y']) else x['diem_x'], axis=1)
diem_term_2_3 = diem_term_2_3.drop(columns=['diem_x', 'diem_y'])
diem_term_2_3 = diem_term_2_3.rename(columns={'gioitinh_x': 'gioitinh', 'khoa_x': 'khoa', 'hedt_x': 'hedt', 'nganhhoc_x': 'nganhhoc', 'sotc_x': 'sotc', 'trangthai_x': 'trangthai', 'hocky_x': 'hocky'})
diem_df = pd.concat([diem_term_1, diem_term_2_3])

### Mở rộng dữ liệu

In [16]:
subject_categories = pd.read_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Augmented/Danh-muc-mon-hoc.xlsx")
subject_description = pd.read_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Augmented/paraphrased_tomtat.xlsx")

In [17]:
augmented_df = pd.merge(filtered_df, subject_categories, left_on='mamh', right_on='Mã MH', how='left')
augmented_df = augmented_df[list(filtered_df.columns) + ['Đơn vị quản lý chuyên môn', 'Loại MH']]

In [ ]:
augmented_df = augmented_df.rename(columns={'Đơn vị quản lý chuyên môn': 'nganhmh', 'Loại MH': 'loaimh'})

In [ ]:
augmented_df = pd.merge(augmented_df, subject_description, on='mamh', how='left')
augmented_df = augmented_df[list(filtered_df.columns) + ['nganhmh', 'loaimh', 'tomtat', 'Paraphrased_Text']]

In [ ]:
augmented_df = augmented_df.rename(columns={'Paraphrased_Text': 'paraphased_tomtat'})

### Lọc dữ liệu

In [ ]:
filtered_augmented_df = augmented_df[augmented_df['loaimh'] != 'BT']

### Word Embedding dữ liệu tomtat

1. PhoBERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

def phobert_embedding(text):
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
  outputs = model(**inputs)
  embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
  return embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
course_paraphased_tomtat = filtered_augmented_df[['mamh', 'paraphased_tomtat']].drop_duplicates(subset=['mamh']).reset_index(drop=True)

In [ ]:
PhoBERT_paraphased_tomtat = course_paraphased_tomtat.copy()
PhoBERT_paraphased_tomtat['paraphased_tomtat'] = PhoBERT_paraphased_tomtat['paraphased_tomtat'].apply(lambda x: phobert_embedding(str(x)) if not pd.isna(x) else np.zeros((1, 768)))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
PhoBERT_paraphased_tomtat['paraphased_tomtat'] = PhoBERT_paraphased_tomtat['paraphased_tomtat'].apply(lambda x: x[0])

In [ ]:
embedding_dim = PhoBERT_paraphased_tomtat['paraphased_tomtat'][0].shape[0]
column_names = [f'embedding_dim_{i}' for i in range(embedding_dim)]

embedding_df = pd.DataFrame(PhoBERT_paraphased_tomtat['paraphased_tomtat'].to_list(), columns=column_names)

In [ ]:
PhoBERT_paraphased_tomtat = pd.concat([PhoBERT_paraphased_tomtat.drop(columns=['paraphased_tomtat']), embedding_df], axis=1)

### Thuộc tính nâng cao

In [ ]:
start_year_df = filtered_augmented_df.groupby('mssv')['namhoc'].min()

In [ ]:
start_year_augmented_df = pd.merge(filtered_augmented_df, start_year_df, on='mssv')
start_year_augmented_df = start_year_augmented_df.rename(columns={'namhoc_x': 'namhoc', 'namhoc_y': 'namhoc_batdau'})

In [ ]:
term_df = start_year_augmented_df
term_df['sohocky'] = term_df.apply(lambda x: (x['namhoc'] - x['namhoc_batdau']) * 2 + x['hocky'], axis=1)

In [ ]:
group_course_df = term_df.copy()
group_course_df['nhomloaimh'] = group_course_df.apply(lambda x: 'CSN' if x['loaimh'] in ['CSNN', 'CSN', 'CNCS'] else 'CN' if x['loaimh'] in ['CNTC', 'CN', 'ĐA'] else 'TN' if x['loaimh'] in ['CĐTN', 'TTTN'] else x['loaimh'], axis=1)

In [ ]:
pass_course_df = group_course_df.copy()
pass_course_df['hoanthanh'] = pass_course_df['diem'] >= 5
pass_course_df['hoanthanh'] = pass_course_df['hoanthanh'].astype(int)

### Lưu dữ liệu

In [ ]:
processed_student_data = pass_course_df[['mssv', 'gioitinh', 'namhoc_batdau', 'khoa', 'hedt', 'nganhhoc']]
processed_score_data = pass_course_df[['mssv', 'mamh', 'nhomloaimh', 'diem', 'trangthai', 'namhoc', 'hocky', 'sohocky', 'hoanthanh']]
processed_course_data = pass_course_df[['mamh', 'sotc', 'nganhmh', 'loaimh', 'nhomloaimh']]
processed_student_data = processed_student_data.drop_duplicates(subset=['mssv'])
processed_course_data = processed_course_data.drop_duplicates(subset=['mamh'])

In [ ]:
processed_student_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Pre-Processed Data/processed_student.xlsx")
processed_score_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Pre-Processed Data/processed_score.xlsx")
processed_course_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Pre-Processed Data/processed_course.xlsx")
PhoBERT_paraphased_tomtat.to_csv("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Pre-Processed Data/PhoBERT_paraphased_tomtat.csv")

In [ ]:
onehot_encoding_student_data = pd.get_dummies(processed_student_data, columns=['gioitinh', 'khoa', 'hedt', 'nganhhoc'])
onehot_encoding_score_data = pd.get_dummies(processed_score_data, columns=['nhomloaimh', 'trangthai'])
onehot_encoding_course_data = pd.get_dummies(processed_course_data, columns=['nganhmh', 'loaimh', 'nhomloaimh'])

In [ ]:
onehot_encoding_student_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Pre-Processed Data/onehot_student.xlsx")
onehot_encoding_score_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Pre-Processed Data/onehot_score.xlsx")
onehot_encoding_course_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Pre-Processed Data/onehot_course.xlsx")

### Thống kê môn học

1. Độ phổ biến

In [ ]:
subject_popularity = term_df.groupby(['mamh', 'namhoc','sohocky', 'khoa'])['mssv'].count()

In [ ]:
subject_popularity = subject_popularity.reset_index().rename(columns={'mssv': 'sosv'})

In [ ]:
total_students = term_df.groupby(['namhoc','sohocky', 'khoa'])['mssv'].count().reset_index().rename(columns={'mssv': 'tongsosv'})

In [ ]:
subject_popularity = pd.merge(subject_popularity, total_students, on=['namhoc', 'sohocky', 'khoa'], how='left')

In [ ]:
subject_popularity['dophobien'] = subject_popularity['sosv'] / subject_popularity['tongsosv']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
subject_popularity['dophobien_scaled'] = scaler.fit_transform(subject_popularity[['dophobien']])

In [ ]:
subject_popularity = subject_popularity.drop(columns=['dophobien_chuan_hoa'])

In [ ]:
subject_popularity.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Pre-Processed Data/subject_popularity.xlsx")

2. Kết quả học tập

In [ ]:
subject_score = term_df.groupby(['mamh', 'namhoc', 'sohocky', 'khoa'])['diem'].mean()

In [ ]:
subject_score = subject_score.reset_index().rename(columns={'diem': 'diemtb'})

In [ ]:
subject_score['dothanhtich'] = subject_score['diemtb'] / 10

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
subject_score['dothanhtich_scaled'] = scaler.fit_transform(subject_score[['dothanhtich']])

In [ ]:
subject_score.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Pre-Processed Data/subject_score.xlsx")

3. Nhóm môn học

In [ ]:
group_course = group_course_df.groupby(['khoa', 'namhoc', 'sohocky', 'nhomloaimh'])['mamh'].count() / group_course_df.groupby(['khoa', 'namhoc', 'sohocky', 'nhomloaimh'])['mssv'].nunique()

In [ ]:
group_course = group_course.reset_index().rename(columns={0: 'somonhoc'})

In [ ]:
group_course.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Pre-Processed Data/group_course.xlsx")